# Climate Data Analysis, Visualization and FAIRness in Practice - DEMO

## Obtaining the dataset

In [ ]:
import intake
import xarray as xr

esm_file = "catalog/CMIP6_ESM_colletion_file.json"
collection = intake.open_esm_datastore(esm_file)

experiment_id = "ssp585"
source_id="CMCC-CM2-SR5"
variable_id="tas"
table_id="day"

query = dict(
    experiment_id=experiment_id,
    source_id=source_id, 
    variable_id=variable_id,
    table_id=table_id
)
search_result = collection.search(**query)
CMIP6 = xr.open_dataset(search_result.df.path[0], engine='zarr', chunks={})

track_id = CMIP6.attrs['tracking_id'].split(':')[-1]

print(f'https://hdl.handle.net/{track_id}')
import requests
import json
response = requests.get(f'https://hdl.handle.net/api/handles/{track_id}')
print(json.dumps(response.json(), indent=1))

CMIP6.to_netcdf(f'data/CMIP6-{experiment_id}_{source_id}_{variable_id}_{table_id}.nc', engine='netcdf4')

from dask.distributed import LocalCluster, Client

def start_dask_cluster(workers, total_memory, n_threads):
    workers = workers
    total_memory = total_memory
    memory = f'{total_memory//workers}GB'
    dask_cluster = LocalCluster(n_workers=workers, memory_limit=memory, threads_per_worker=n_threads)
    _dask_client = Client(dask_cluster)
    return _dask_client

import xarray as xr
import intake

client = start_dask_cluster(workers=1, total_memory=32, n_threads=8)

catalog = intake.open_catalog('catalog/catalog.yaml')
ds = catalog.cmip6_netcdf.to_dask()
ds = ds.chunk({
    "time": len(ds.time.values)//5,
    "lat": len(ds.lat.values)//4,
    "lon": len(ds.lon.values)//4
})
ds.to_zarr('data/cmip6.zarr', mode='w', consolidated=True)

client.shutdown()

## Loading the new Zarr Dataset

In [ ]:
import xarray as xr
import intake
import hvplot.xarray
import panel as pn
pn.extension("tabulator")

In [ ]:
import dask
from dask.distributed import LocalCluster, Client

In [ ]:
def start_dask_cluster(workers, total_memory, n_threads):
    workers = workers
    total_memory = total_memory
    memory = f'{total_memory//workers}GB'
    dask_cluster = LocalCluster(n_workers=workers, memory_limit=memory, threads_per_worker=n_threads)
    _dask_client = Client(dask_cluster)
    return _dask_client

In [ ]:
from prov_tracking import ProvTracker

client = start_dask_cluster(workers=8, total_memory=32, n_threads=1)
plugin = ProvTracker(rich_types=True)
client.register_plugin(plugin)
plugin.start(client.scheduler)

In [ ]:
catalog = intake.open_catalog('catalog/catalog.yaml')

In [ ]:
zarr = catalog.cmip6_zarr.to_dask()

In [ ]:
zarr

## Compute a Seasonal Climatology

Seasonal climatology refers to the analysis of long-term average atmospheric and surface conditions (such as temperature, precipitation, and wind) computed separately for each of the four meteorological seasons (e.g., DJF, MAM, JJA, SON). By averaging data over many years, it highlights the characteristic “typical” climate for each season and helps identify deviations or trends relative to those seasonal norms.

This kind of aggregation can be performed using XArray built-in function [groupby](https://docs.xarray.dev/en/stable/user-guide/groupby.html#groupby)

In [ ]:
seasonal_climatology = zarr.tas.groupby("time.season").mean() - 273.15

In [ ]:
seasonal_climatology

In [ ]:
%%time
seasonal_climatology = seasonal_climatology.compute()

In [ ]:
seasonal_climatology

In [ ]:
season = pn.widgets.Select(name='Season', options=list(seasonal_climatology.season.values))
seasonal_climatology.interactive().sel(season=season).hvplot(cmap='seismic')

### 🔍 Seasonal Anomaly Calculation

To analyze climate variability, it is common to compare current observations with long-term averages. A **seasonal anomaly** measures the difference between observed seasonal conditions and the corresponding climatological mean.

In this case, we compute the temperature anomaly for the **summer season (JJA) of a given year** using the following steps:

1. **Extract data for the summer period** (June–August) of the target year from the hourly dataset.
2. **Aggregate the seasonal data** by computing the mean temperature across the selected months.
3. **Select the climatological mean** for the summer season (JJA) from a precomputed seasonal climatology.
4. **Subtract the climatology from the seasonal mean**, yielding the anomaly field:

$$
\text{Anomaly}_{\text{year},\text{JJA}} = \overline{T}_{\text{year},\text{JJA}} - \overline{T}_{\text{Climatology},\text{JJA}}
$$

This anomaly highlights regions where temperatures during the summer were **above or below normal**, providing insight into unusual climate behavior or extreme events.


In [ ]:
import cartopy.crs as ccrs 

In [ ]:
seasonal_climatology = zarr.tas.sel(time=slice('2015','2025')).groupby("time.season").mean() - 273.15

In [ ]:
summer = zarr.sel(time=slice("2025-06-01", "2025-08-31"))

In [ ]:
summer_mean = summer.tas.mean(dim='time') - 273.15
summer_mean = summer_mean.compute()

In [ ]:
jja_climatology = seasonal_climatology.sel(season='JJA')

In [ ]:
summer_anomaly = summer_mean - jja_climatology

In [ ]:
summer_anomaly

In [ ]:
dask.visualize(summer_anomaly, filename='summer_dask_graph', format='svg')

In [ ]:
summer_anomaly.interactive().hvplot(cmap='coolwarm', crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree())

### Generate the provenance graph

In [ ]:
client.shutdown()